# 7장 딥러닝을 위한 고급 도구

## 7.1 Sequential 모델을 넘어서: 케라스의 함수형 API

Sequential 모델은 네트워크 입력과 출력이 하나라고 가정합니다. 이 모델은 층을 차례대로 쌓아 구성합니다.  

많은 경우에 이런 가정이 적절합니다. 하지만 일부 네트워크는 개별 입력이 여러 개 필요하거나 출력이 여러 개 필요합니다. 층을 차례대로 쌓지 않고 층 사이를 연결하여 그래프처럼 만드는 네트워크도 있습니다.

최근에 개발된 많은 신경망 구조는 선형적이지 않은 네트워크 토폴로지가 필요합니다.
예를 들어 (구글의 세게디 등이 개발한) 인셉션 모듈을 사용하는 인셉션 계열의 네트워크들입니다. 이 모듈에서 입력은 나란히 놓인 여러 개의 합성곱 층을 거쳐 하나의 텐서로 출력이 합쳐집니다.

모델에 잔차 연결을 추가하는 경향도 있습니다. ResNet 계열의 네트워크들이 이런 방식을 사용하기 시작했습니다. 잔차 연결은 하위 층의 출력 텐서를 상위 층의 출력 텐서에 더해서 아래층의 표현이 네트워크 위쪽으로 흘러갈 수 있도록 합니다. 하위 층에서 학습된 정보가 데이터 처리 과정에서 손실되는 것을 방지합니다.

여러 경우에 다중 입력 모델, 다중 출력 모델, 그래프 구조를 띤 모델이 필요하지만 케라스의 Sequential 클래스를 사용해서는 만들지 못합니다.

케라스에는 훨씬 더 일반적이고 유연한 다른 방법인 함수형 API가 있습니다. 이 절에서 함수형 API가 무엇인지 소개하고, 함수형 API를 사용하는 방법과 이를 사용하여 할 수 있는 것을 자세히 설명하겠습니다.

In [ ]:
from keras import Input, layers

input_tensor = Input(shape=(32,))
dense = layers.Dense(32, activation='relu')

output_tensor = dense(input_tensor)

In [ ]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

# seq_model = Sequential()
# seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
# seq_model.add(layers.Dense(32, activation='relu'))
# seq_model.add(layers.Dense(10, activation='softmax'))


input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

# 입력과 출력 텐서를 지정하여 Model 클래스의 객체를 만듭니다.
model = Model(input_tensor, output_tensor)

model.summary()



Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


### 7.1.2 다중 입력 모델

함수형 API는 다중 입력 모델을 만드는 데 사용할 수 있습니다. 일반적으로 이런 모델은 서로 다른 입력 가지를 합치기 위해 여러 텐서를 연결할 수 있는 층을 사용합니다. 

이와 관련된 케라스의 함수는 keras.layers.add, keras.layers.concatenate 등 입니다. 

In [ ]:
from keras.models import Model
from keras import layers
from keras import Input

text_volcabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')

# 입력을 크기가 64인 벡터의 시퀀스로 임베딩합니다.
embedded_text = layers.Embedding(text_volcabulary_size, 64)(text_input)

# LSTM을 사용하여 이 벡터들을 하나의 벡터로 인코딩합니다.
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question')

embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

# 인코딩된 질문과 텍스트를 연결합니다.
concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

# 소프트맥스 분류기를 추가합니다.
answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

# 모델 객체를 만들고 2개의 입력과 출력을 주입합니다.
model = Model([text_input, question_input], answer)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])